# Asbestos Audit:
### outputs:
* ASB audit
* Duplicate APNs in share point, Tetra forms, Smart Sheets

In [1]:
import pandas as pd
import glob
import numpy as np
import os

Variables

In [2]:
county = ['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY', 'SHASTA']


file_type_ex = '.xlsx'
file_type_csv = '.csv'

#  Smart Sheets
## Gets our most recent dowloaded files from the downloads folder

In [3]:
# read in BOP Smart hseets tracker
path1 = r"C:\Users\jacque.trahan\Downloads\BOP_North Div 2021 Debris Removal Tracker*"
load_it1 = glob.glob(path1 + file_type_ex)

df1 = pd.read_excel(max(load_it1, key=os.path.getctime), parse_dates=True)

In [4]:
# read in ANVIL Smart sheeets tracker
path_s = r"C:\Users\jacque.trahan\Downloads\ANVIL_North Div 2021 Debris Removal Tracker*"
load_it_s = glob.glob(path_s + file_type_ex)

df_s = pd.read_excel(max(load_it_s, key=os.path.getctime), parse_dates=True)

### Concat Northern and south cental files

In [5]:
df_both = pd.concat([df1, df_s])

In [6]:
# Read in asbestos tracker from share point
path2 = r"C:\Users\jacque.trahan\Downloads\Asbestos Tracker*"

load_it2 = glob.glob(path2 + file_type_csv)

df2 = pd.read_csv(max(load_it2, key=os.path.getctime))

In [7]:
# df2.info()

In [8]:
# read in ESL from SQL server
path3 = r"C:\Users\jacque.trahan\Downloads\Asbestos Sample Status*"
load_it3 = glob.glob(path3 + file_type_ex)
esl_final = pd.read_excel(max(load_it3, key=os.path.getctime), header=1)

Tetra forms has a key error that we have to open the file first then save it as a new file. 
Only then will it all me to read the file in pandas
* KeyError: "There is no item named 'xl / sharedStrings.xml' in the archive"

# Tetra Forms

In [9]:
df3= pd.read_excel("../../tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx")

# load in ESL with Concat
### No longer needed. But here for referacne on how to concat all files in a folder.
#### New way has master file and can just be read from downloads. 
* reads all files in a folder


In [10]:
# path = r"C:\Users\jacque.trahan\Desktop\Dixie Fires Audit\ESLs\Northern"

# esl_final = pd.concat(map(pd.read_excel, glob.glob(path + '/*.xlsx')))




* Below is What the above is doing 
* The above way is faster
* no list

path = r"C:\Users\jacque.trahan\Desktop\Dixie Fires Audit\ESLs\Northern"
esl_final= glob.glob(path + "/*.xlsx")

li=[]
for filename in all_files:
    df = pd.read_excel(filename, index_col=None)
    li.append(df)

esl_final = pd.concat(li)

In [11]:
smartsheets = df_both.copy()
asbestos = df2.copy()
tetratForms = df3.copy()

# Most important info

What I need to Check Daily is Smart Sheets vs ENV. Trackers
* Chimneys and neshaps match
* FINAL RESULTS vs Smart sheets – biggest concern

* ASB Assessment Date Match

* End of day numbers match

# Asbestos set up

In [12]:
asbestos['Division'].value_counts()

Northern Division    1185
Central Division     1047
Name: Division, dtype: int64

In [13]:
# Take out all divisions other than Northern.
asbestos = asbestos[asbestos['Division'].isin(['Northern Division'])]

###  Get APN counts from Asbetsos

In [14]:
# pd.set_option('display.max_rows',1000)

In [15]:
asbestos['APN'].value_counts().to_excel('Asbestos tracker APN Counts_Share Point-northern.xlsx')

In [16]:
# changes date column from object to a date.
asbestos['Date Collected'] = pd.to_datetime(asbestos['Date Collected']).dt.date.fillna(0)
asbestos['Chimney Tip Date'] = pd.to_datetime(asbestos['Chimney Tip Date']).dt.date.fillna(0)
asbestos['Chimney Abatement Completed Date'] = pd.to_datetime(asbestos['Chimney Abatement Completed Date']).dt.date.fillna(0)
asbestos['Asbestos Abatement Completed Date'] = pd.to_datetime(asbestos['Asbestos Abatement Completed Date']).dt.date.fillna(0)
asbestos['Date Shipped or Driven'] = pd.to_datetime(asbestos['Date Shipped or Driven']).dt.date.fillna("0")

In [17]:
asbestos.columns = map(str.upper, asbestos.columns)

In [18]:
asbestos.rename(columns={'APN':'APN_ASB'}, inplace=True)

In [19]:
# Column section all upper case
asb_cols = asbestos[['APN_ASB', 'FINAL RESULTS', 'CHIMNEY', 'CHIMNEY TIP DATE', 'CHIMNEY FINDING',
                 'NESHAP WALLS',
                 'DATE COLLECTED', 'DATE SHIPPED OR DRIVEN', 
                 'SAMPLE QUANTITY', 'POINT COUNT NEEDED', 'POINT COUNT RESULTS',
                 'ASBESTOS ABATEMENT COMPLETED DATE', 'CHIMNEY ABATEMENT COMPLETED DATE',
                 'PLANNING NOTES', 'DIVISION']]

# Tetra Forms setup

### Tetra Forms sheet sample_log setup

In [20]:
# read in the sample log of the survey forms to get the correct sample count
TF = pd.read_excel("../../tetra forms ASB and SA/ASB_TetraForms_survey1.xlsx", sheet_name='sample_log')

In [21]:
# will grab the highest sample taken
max_samples = TF.groupby(['apn'])[['samplecounter']].max().reset_index()


## Tetra Forms chimney tip setup
Combine chimney tip database with sample log

In [22]:
# Read in the chuimney tip database
TF_tips = pd.read_excel('../../tetra forms ASB and SA/ASB_TetraForms_tips1.xlsx')

In [23]:
TF_tips = TF_tips[TF_tips['county'].isin(county)]

In [24]:
# columns needed
TF_tips= TF_tips[['apn', 'inspection_date']]

### Get Chimney Tip APN counts in asbestos

In [25]:
TF_tips['apn'].value_counts().to_excel('Asbestos Chimney tip APN counts.xlsx') # 1 for 1 last check all good

In [26]:
# rename for easy reading for later merges
TF_tips.rename(columns={'inspection_date': 'TIP INSPECTION_TF'}, inplace=True)

In [27]:
# Need to make this series a datetime and fill in 0'sfor blank spots
TF_tips['TIP INSPECTION_TF'] = pd.to_datetime(TF_tips['TIP INSPECTION_TF']).dt.date.fillna(0)

In [28]:
# merge tips and sample log databases
merge1 = max_samples.merge(TF_tips,
                          left_on='apn',
                          right_on='apn',
                          how='outer')

# Tetra Forms Abatment date setup
Combine Abatment date database with merge 1 (tips and max samples)

In [29]:
# abatment clearance form logs
TF_abate = pd.read_excel('../../tetra forms ASB and SA/ASB_TetraForms_abatment1.xlsx')

In [30]:
# chossing couty for only Northern division. Only way to seperate the divisions in Tetra forms
TF_abate = TF_abate[TF_abate['county'].isin(county)]

In [31]:
# columns needed
TF_abate = TF_abate[['apn', 'abatement_end']]

#### Get Tetra Forms Abatment APN counts in asbestos

In [32]:
TF_abate['apn'].value_counts().to_excel('TF Abatmanet APN Counts Northern.xlsx')

In [33]:
# Need to make this series a datetime and fill in 0'sfor blank spots
TF_abate['abatement_end'] = pd.to_datetime(TF_abate['abatement_end']).dt.date.fillna(0)

In [34]:
# rename for easy reading on later merges
TF_abate.rename(columns={'abatement_end': 'ABATEMENT END_TF'}, inplace=True)

In [35]:
merge2 = merge1.merge(TF_abate,
                     left_on='apn',
                     right_on='apn',
                     how='outer')

# Master Tetra Forms setup from all thre databases
* Chimney tips
* abatement dates
* asbestos survey info

In [36]:
# merges in tetra froms surveys with sample logs, tips, and abataemetn databses.
tetratForms = tetratForms.merge(merge2,
                 left_on='apn',
                 right_on='apn',
                 how='outer')

In [37]:
# chossing couty for only Northern division. Only way to seperate the divisions in Tetra forms
tetratForms = tetratForms[tetratForms['county'].isin(county)]

In [38]:
# columns needed
tetratForms = tetratForms[['apn', 'chimneys','walls','inspection_date', 'county','samplecounter','TIP INSPECTION_TF',
                           'ABATEMENT END_TF' ]]

###  TF Survey APN counts

In [39]:
tetratForms['apn'].value_counts().to_excel('Asbestos Survey TF_APN counts Northern.xlsx')

In [40]:
# change date object to date time format
tetratForms['inspection_date'] = pd.to_datetime(tetratForms['inspection_date']).dt.date.fillna(0)

In [41]:
# rename out columns for easier readin on merges
tetratForms.rename(columns={'inspection_date': 'survey inspection date'}, inplace=True)

In [42]:
# make ALL columnS upper CASE
tetratForms.columns =map(str.upper, tetratForms.columns)

In [43]:
tetratForms.rename(columns={'APN': 'APN_TF'}, inplace=True)

# Smart Sheet set up

### Get APN counts from Smart Sheets

In [44]:
smartsheets['APN_ROW Segment'].value_counts().to_excel('Northern_SS Tracker APN counts.xlsx')

In [45]:
smartsheets["Chimney Tipped"].fillna(0, inplace=True)


#### This is need becasue BUG: pd.to_datetime() throws if caching is on with Null-like arguments
* solution: pd.to_datetime(df.colx.fillna(pd.NaT), errors='coerce')

In [46]:
smartsheets["Chimney Tipped"] = pd.to_datetime(smartsheets["Chimney Tipped"].fillna(pd.NaT), errors='coerce')
smartsheets["ASB Assessment"] = pd.to_datetime(smartsheets["ASB Assessment"].fillna(pd.NaT), errors='coerce')
smartsheets["ASB Abatement"] = pd.to_datetime(smartsheets["ASB Abatement"].fillna(pd.NaT), errors='coerce')

In [47]:
# changes date column from object to a date. and fill na with 0's
smartsheets["ASB Assessment"] = smartsheets["ASB Assessment"].dt.date.fillna(0)
smartsheets["Chimney Tipped"] = smartsheets["Chimney Tipped"].dt.date.fillna(0)
smartsheets["ASB Abatement"] = smartsheets["ASB Abatement"].dt.date.fillna(0)

In [48]:
smartsheets.rename(columns={'ASB Assessment': 'Date Collected', 'Chimney Tipped':'CHIMNEY TIP DATE',
                            'County': 'SS_COUNTY'}, inplace=True)

In [49]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [50]:
# Column section normal uppercase
ss_cols = smartsheets[['APN_ROW SEGMENT','STREET #','STREET NAME','STRUCTURAL STATUS','SS_COUNTY','ASB RESULTS',
                     'CHIMNEY','CHIMNEY TIP DATE',
                     'NESHAP WALLS', 
                     "DATE COLLECTED",'ASB SUSPECTED',
                     'ASB ABATEMENT', 'DIVISION']]
ss_cols = ss_cols[ss_cols['DIVISION'].isin(['Northern Division'])]

# Smasrt merge first w/ ASB

In [51]:
smasrt_ASB = ss_cols.merge(asbestos,
                           left_on='APN_ROW SEGMENT',
                           right_on='APN_ASB',
                           how='outer',
                         suffixes=('_SS','_ASB'))

# smartsheet w/ ASB merge with Tetraform

In [52]:
SS_ASB_TF = smasrt_ASB.merge(tetratForms,
           left_on="APN_ROW SEGMENT",
           right_on='APN_TF',
           how='outer',
           suffixes=("_SS", "_TF"))

# ESL Set up

In [53]:
esl_final.columns = map(str.upper, esl_final.columns)

In [54]:
esl_final.rename(columns={'DATE SHIPPED':'DATE SHIPPED_esl'}, inplace=True)

In [55]:
esl_final = esl_final[esl_final['DIVISION'].isin(['Northern Division'])]

In [56]:
exceedances = esl_final[['APN', 'BULK PLM EXCEEDANCES']]

In [57]:
esl_final = esl_final[['APN', 'DATE SHIPPED_esl','BULK PLM EXCEEDANCES', 'DIVISION']]

In [58]:
esl_final['DATE SHIPPED_esl'] = pd.to_datetime(esl_final['DATE SHIPPED_esl']).dt.date.fillna("0")

In [59]:
esl = esl_final.groupby(['APN'])[['DATE SHIPPED_esl']].max().reset_index()

# Merge SS_ASB_TF Merge with ESL master

In [60]:
pre_final = SS_ASB_TF.merge(esl,
            left_on="APN_ROW SEGMENT",
            right_on='APN',
            how='outer')

In [61]:
final = pre_final.merge(exceedances,
                       left_on="APN_ROW SEGMENT",
                       right_on='APN',
                       how='outer')

In [62]:
# final.columns

# set up columns for QC order

In [63]:
# change the ND/POS to spell out for easier matching in audits
final['FINAL RESULTS'] = final['FINAL RESULTS'].str.replace("ND", 'NEGATIVE').str.replace("POS","POSITIVE")

# Get hacky and force blanks to say 0
this is beacsue some APN are not in every tracker yet

## Smartsheets 


In [64]:
# final.columns

In [65]:
final["DATE COLLECTED_SS"] = final["DATE COLLECTED_SS"].fillna(0)

In [66]:
final['CHIMNEY TIP DATE_SS'] = final['CHIMNEY TIP DATE_SS'].fillna(0)
final['ASB RESULTS'] = final['ASB RESULTS'].fillna("0")

final['CHIMNEY_SS'] = final['CHIMNEY_SS'].str.replace("No", '0')
final['CHIMNEY_SS'] = final['CHIMNEY_SS'].fillna("0")

final["ASB ABATEMENT"] = final["ASB ABATEMENT"].fillna(0)

final["NESHAP WALLS_SS"] = final["NESHAP WALLS_SS"].str.replace("No", "0").fillna("0")
final['ASB SUSPECTED'] = final['ASB SUSPECTED'].fillna("0")


## ASB Tracker


In [67]:
final['DATE COLLECTED_ASB'] = final['DATE COLLECTED_ASB'].fillna(0)
final['DATE SHIPPED OR DRIVEN'] = final['DATE SHIPPED OR DRIVEN'].fillna("0")

final['CHIMNEY TIP DATE_ASB'] = final['CHIMNEY TIP DATE_ASB'].fillna(0)

final['SAMPLE QUANTITY'] = final['SAMPLE QUANTITY'].fillna(0)

final['FINAL RESULTS'] = final['FINAL RESULTS'].fillna("0")
final['POINT COUNT RESULTS'] = final['POINT COUNT RESULTS'].fillna("0")

final['CHIMNEY_ASB'] = final['CHIMNEY_ASB'].str.replace("No", "0")
final['CHIMNEY_ASB'] = final['CHIMNEY_ASB'].fillna('0')

final["ASBESTOS ABATEMENT COMPLETED DATE"] = final["ASBESTOS ABATEMENT COMPLETED DATE"].fillna(0)

final["NESHAP WALLS_ASB"] = final["NESHAP WALLS_ASB"].str.replace("No", "0").fillna("0")


## Tetra Forms


In [68]:
final['TIP INSPECTION_TF'] = final['TIP INSPECTION_TF'].fillna(0)

final['SURVEY INSPECTION DATE'] = final['SURVEY INSPECTION DATE'].fillna(0)

final['ABATEMENT END_TF'] = final['ABATEMENT END_TF'].fillna(0)

final['SAMPLECOUNTER'] = final['SAMPLECOUNTER'].fillna(0)

final['CHIMNEYS'] = final['CHIMNEYS'].str.replace('No', "0").fillna("0")


final["ABATEMENT END_TF"] = final["ABATEMENT END_TF"].fillna(0)

final["WALLS"] = final["WALLS"].str.replace("No", "0").fillna("0")


# ESL fillna with 0's

In [69]:
final['DATE SHIPPED_esl'] = final['DATE SHIPPED_esl'].fillna("0")
final['DATE SHIPPED OR DRIVEN'] = final['DATE SHIPPED OR DRIVEN'].fillna("0")

In [70]:
# final.columns

In [71]:
final = final[["APN_ROW SEGMENT",'APN_ASB','APN_TF','STREET #','STREET NAME_SS','STRUCTURAL STATUS','SS_COUNTY',
               "DATE COLLECTED_SS",'DATE COLLECTED_ASB','SURVEY INSPECTION DATE',
               'SAMPLE QUANTITY','SAMPLECOUNTER',
               'ASB SUSPECTED', 'DATE SHIPPED OR DRIVEN','DATE SHIPPED_esl',
               'ASB RESULTS','FINAL RESULTS','POINT COUNT NEEDED', 'POINT COUNT RESULTS', 'BULK PLM EXCEEDANCES',
               'CHIMNEY_SS','CHIMNEY_ASB','CHIMNEYS',
               'CHIMNEY TIP DATE_SS','CHIMNEY TIP DATE_ASB','TIP INSPECTION_TF',
               'CHIMNEY FINDING', 'CHIMNEY ABATEMENT COMPLETED DATE',
               'ASB ABATEMENT','ASBESTOS ABATEMENT COMPLETED DATE', 'ABATEMENT END_TF',
               'NESHAP WALLS_SS','NESHAP WALLS_ASB','WALLS',
               'DIVISION_SS']]

# If ASB is suspected in SS shipped should not be blank in ASB tracker

In [72]:
def shipping_dates(df, comp1, comp2, comp3):
    if ((df[comp1] != "0") & ((df[comp2] == "0") | (df[comp3] == "0"))):
        return 0
    else:
        return 1

In [73]:
final.insert(final.columns.get_loc('DATE SHIPPED_esl')+1, 'Blank Ship Date',
            final.apply(shipping_dates,comp1='ASB SUSPECTED', comp2='DATE SHIPPED OR DRIVEN', comp3='DATE SHIPPED_esl',axis=1))

# Automate the Excel QC process Function for all below process

In [74]:
def compare_cols(df, comp1, comp2, comp3=None):
    if comp3 == None:
        if df[comp1] == df[comp2]:
            return 1
        else:
            return 0
    elif ((df[comp1] == df[comp2]) & (df[comp1] == df[comp3]) & (df[comp2] == df[comp3])):
        return 1
    else:
        return 0

# Check date shipped in ASB and date shipped in ESL

In [75]:
final.insert(final.columns.get_loc('Blank Ship Date')+1, 'Check if ship dates Match',
             final.apply(compare_cols, comp1='DATE SHIPPED OR DRIVEN', comp2='DATE SHIPPED_esl', axis=1))

# check Date Collected

In [76]:
final.insert(final.columns.get_loc('SAMPLE QUANTITY')+1,'Date Collected - M',
             final.apply(compare_cols, comp1="DATE COLLECTED_SS", comp2='DATE COLLECTED_ASB', comp3='SURVEY INSPECTION DATE', axis=1))

# check Total Samples Collected Counts

In [77]:
final.insert(final.columns.get_loc('SAMPLECOUNTER')+1, "Total Samples Collected Counts-M", 
            final.apply(compare_cols, comp1='SAMPLE QUANTITY', comp2='SAMPLECOUNTER',axis=1))

In [78]:
# have to use in place here or it will turn everythig to 0 if we use df[cols] = df[col].fillna(0) Like above
final['Total Samples Collected Counts-M'].fillna(0, inplace=True)

# check Final Results

Not working correctly. leaves a blank if there is an NS in column. Solve it

In [79]:
final.insert(final.columns.get_loc('BULK PLM EXCEEDANCES')+1, "Final Results-M", 
            final.apply(compare_cols, comp1="ASB RESULTS", comp2="FINAL RESULTS", axis=1))

In [80]:
final["Final Results-M"].fillna(0,inplace=True)

# Check Chimney 

In [81]:
final.insert(final.columns.get_loc('CHIMNEYS')+1, 'Has Chimney-M',
            final.apply(compare_cols, comp1="CHIMNEY_SS", comp2="CHIMNEY_ASB", comp3="CHIMNEYS", axis=1))

In [82]:
final['Has Chimney-M'].fillna(0, inplace=True)

# check CHIMNEY TIP DATE

In [83]:
final.insert(final.columns.get_loc('TIP INSPECTION_TF')+1, "Chimney Tip dates-M", 
            final.apply(compare_cols, comp1="CHIMNEY TIP DATE_SS", comp2="CHIMNEY TIP DATE_ASB", comp3="TIP INSPECTION_TF", axis=1))

# check Abatements Dates

In [84]:
final.insert(final.columns.get_loc('ABATEMENT END_TF')+1, "ASB ABATEMENT Date-M",
            final.apply(compare_cols,
                        comp1="ASB ABATEMENT", comp2="ASBESTOS ABATEMENT COMPLETED DATE", comp3="ABATEMENT END_TF", axis=1))

# check NESHAPS

In [85]:
final.insert(final.columns.get_loc('WALLS')+1, "Neshap Wall-M",
            final.apply(compare_cols, comp1="NESHAP WALLS_SS", comp2="NESHAP WALLS_ASB", comp3="WALLS", axis=1))

# Highight columns

In [86]:
# # set match columns to a certain color:
# # SYNTAX - df.style.set_properties(**{'background-color': 'color wanted'}, subset=["Col A", 'Col b', 'etc.'])

df_style = final.style.set_properties(**{'background-color': '#ffffb3'},
                           subset=['Date Collected - M',"Neshap Wall-M","ASB ABATEMENT Date-M",
                                  "Chimney Tip dates-M",'Has Chimney-M',
                                   "Final Results-M","Total Samples Collected Counts-M",
                                   'Blank Ship Date', 'Check if ship dates Match'], inplace=True)

# Make Smart sheets APNs the index and put in excel

In [87]:
# final.set_index("APN_ROW SEGMENT", inplace=True)

In [88]:
df_style.to_excel("Northern Dixie ASB Audit.xlsx")

# ASB sample sent prior to ASB assessment

In [95]:
asb_sample = df_both.copy()
asb_sample = asb_sample[['APN_ROW Segment','Street #', 'Street Name', 'County',
                         'ASB Assessment', 'ASB Sample Sent']]
asb_sample = asb_sample[(asb_sample['ASB Assessment'] > asb_sample['ASB Sample Sent'])]

asb_sample.set_index('APN_ROW Segment', inplace=True)
asb_sample.to_excel('ASB sample sent prior to ASB assessment.xlsx')

# ASB results prior to ASB sample sent date

In [90]:
asb_res = df_both.copy()
asb_res = asb_res[['APN_ROW Segment','Street #', 'Street Name', 'County',
                   'ASB Sample Sent','ASB Results Received', 'ASB Results']]
asb_res = asb_res[(asb_res['ASB Sample Sent'].isna()) & (asb_res['ASB Results'].notna())]


asb_res.set_index('APN_ROW Segment', inplace=True)
asb_res.to_excel('ASB results prior to ASB sample sent date.xlsx')

# Get number of ssamples taken compared to number of samples positive.
### this is extra if I have time.

In [91]:
# final[['FINAL RESULTS']].value_counts().plot(kind='pie',
#                                            labels=['No Data', "Negitives", "Positives", "NS"],
#                                              colors = ['yellow', 'green', 'red', 'lightblue'],
#                                           autopct='%.2f',
#                                            fontsize=15,
#                                           figsize=(6,6),
#                                            title=' ASB Overview by %',
#                                             ylabel="")



In [92]:
# final[['ASB RESULTS']].value_counts().plot(kind='pie',
#                                            labels=['No Data', "Negitives", "Positives"],
#                                            colors = ['yellow', 'green', 'red'],
#                                           autopct='%.2f',
#                                            fontsize=15,
#                                           figsize=(6,6),
#                                            title='Smart Sheets ASB Overview by %',
#                                           ylabel="")

In [93]:
# plt.figure(figsize=(12,6))
# sns.countplot(data=final, x='SS_COUNTY', hue='ASB RESULTS')


In [94]:
# plt.figure(figsize=(12,6))
# sns.countplot(data=final, x='SS_COUNTY', hue='FINAL RESULTS')